In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras import callbacks
from keras.regularizers import l2
import pickle
import numpy as np
import datetime
import os
import random
import gc
now_datetime  = datetime.datetime.now()
NAME = f"#Face_spoofing300x2_{now_datetime.day:02d}{now_datetime.month:02d}{now_datetime.year}_{now_datetime.hour:02d}{now_datetime.minute:02d}"
dir_pickle = "database_serialized"
dir_models_save = "models"
ignore = []

Using TensorFlow backend.


In [2]:
format2 = [[120,250,400],[200,400]]
format3 = [[80,120,300],[400]]
NN_formats = [format2,format3]

In [3]:
def load_data(person):   
    pickle_path =os.path.join(dir_pickle,person)
    pickle_in = open(os.path.join(pickle_path,f"X{person}.pickle"),"rb")
    X = pickle.load(pickle_in)
    pickle_in.close()
    pickle_in = open(os.path.join(pickle_path,f"y{person}.pickle"),"rb")
    y = pickle.load(pickle_in)
    pickle_in.close()
    
    X = X.astype(np.float16)/255.0
    
    return X, y

def create_model(person, pickle_path,format_convolution,format_dense):
    model = Sequential()
    is_first = True
    # Convolutional layers
    for format_c in format_convolution:
        if is_first:
            model.add(layers.Conv2D(format_c,(3,3),input_shape=[300,300,3]))
            is_first = False
        else:
            model.add(layers.Conv2D(format_c,(3,3)))
        model.add(layers.Activation("relu"))
        model.add(layers.MaxPool2D(pool_size=(3,3)))
    #Flatten the model if needed
    model.add(layers.Flatten())
    
    #Dense layers
    for format_d in format_dense:
        model.add(layers.Dropout(0.12))
        model.add(layers.Dense(format_d,activation="relu",kernel_regularizer=l2(0.002)))
    
    #Output layer
    model.add(layers.Dense(1,activation="sigmoid"))
    
    
    model.compile(loss="binary_crossentropy",
              optimizer = "adam",
              metrics= ["accuracy"])
    
    return model

In [4]:
def create_callbacks(person,format_convolution,format_dense):
    if not os.path.isdir(os.path.join(dir_models_save,person)):
        os.mkdir(os.path.join(dir_models_save,person))
    checkpoints = callbacks.ModelCheckpoint(filepath=os.path.join(dir_models_save,person,f"{person}{NAME}{format_convolution}{format_dense}.h5"),
                                                monitor="val_acc",
                                                mode = "max",
                                                verbose = 1,
                                                save_weights_only=False,
                                                save_best_only=True)
    return checkpoints
        
def create_validation(person,val_test_split=0.25):
    ValSet = []
    pickle_path =os.path.join(dir_pickle,person)
    pickle_in = open(os.path.join(pickle_path,f"X{person}Test.pickle"),"rb")
    Xval = pickle.load(pickle_in)
    pickle_in.close()
    pickle_in = open(os.path.join(pickle_path,f"y{person}Test.pickle"),"rb")
    yval = pickle.load(pickle_in)
    pickle_in.close()
    Xval = Xval.astype(np.float16)
    Xval = Xval/255.0
    yval = yval
    print(Xval.shape[0])
    n_exs = int(Xval.shape[0] * val_test_split)
    print(n_exs)
    ValSet = [np.array(Xval[:n_exs-1]),yval[:n_exs-1]]
    return ValSet

In [5]:
people = os.listdir(dir_pickle)
models = []
ValSet = create_validation("_All",val_test_split=0.30)
for conv, dense in NN_formats:
    models.append(create_model("_All",os.path.join(dir_pickle,"_All"),conv,dense))
    checkpoint = create_callbacks("_All",conv,dense)
for model in models:
    for person in people:
        if person == "_All":
            continue
        X, y = load_data(person)
        model.fit(X,y,batch_size=7,validation_data=ValSet,epochs=3,shuffle=False, callbacks=[checkpoint])
        del X,y
        gc.collect()
    

_All
3952
1185
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 10635 samples, validate on 1184 samples
Epoch 1/3
10633/10635 [============================>.] - ETA: 0s - loss: 0.1395 - acc: 0.9865
Epoch 00001: val_acc improved from -inf to 0.58108, saving model to models\_All\_All#Face_spoofing300x2_17102019_0301[80, 120, 300][400].h5
10635/10635 [==============================] - 265s 25ms/sample - loss: 0.1394 - acc: 0.9865 - val_loss: 5.8601 - val_acc: 0.5811
Epoch 2/3
10633/10635 [============================>.] - ETA: 0s - loss: 0.0600 - acc: 0.9952
Epoch 00002: val_acc did not improve from 0.58108
10635/10635 [==============================] - 252s 24ms/sample - loss: 0.0600 - acc: 0.9952 - val_loss: 11.7872 - val_acc: 0.5203
Epoch 3/3
10633/10635 [============================>.] - ETA: 0s - loss: 0.0241 - acc

Epoch 2/3
2779/2785 [============================>.] - ETA: 0s - loss: 0.0766 - acc: 0.9906
Epoch 00002: val_acc did not improve from 0.75084
2785/2785 [==============================] - 73s 26ms/sample - loss: 0.0766 - acc: 0.9907 - val_loss: 3.8831 - val_acc: 0.5988
Epoch 3/3
2779/2785 [============================>.] - ETA: 0s - loss: 0.0498 - acc: 0.9968
Epoch 00003: val_acc did not improve from 0.75084
2785/2785 [==============================] - 73s 26ms/sample - loss: 0.0498 - acc: 0.9968 - val_loss: 2.5038 - val_acc: 0.7002
Train on 2785 samples, validate on 1184 samples
Epoch 1/3
2779/2785 [============================>.] - ETA: 0s - loss: 0.1790 - acc: 0.9867
Epoch 00001: val_acc did not improve from 0.75084
2785/2785 [==============================] - 46s 17ms/sample - loss: 0.1788 - acc: 0.9867 - val_loss: 4.4340 - val_acc: 0.6402
Epoch 2/3
2779/2785 [============================>.] - ETA: 0s - loss: 0.0425 - acc: 1.0000- ETA: 2s - lo
Epoch 00002: val_acc did not improve fr

Epoch 3/3
3073/3078 [============================>.] - ETA: 0s - loss: 0.0030 - acc: 1.0000
Epoch 00003: val_acc improved from 0.77449 to 0.77956, saving model to models\_All\_All#Face_spoofing300x2_17102019_0301[80, 120, 300][400].h5
3078/3078 [==============================] - 55s 18ms/sample - loss: 0.0030 - acc: 1.0000 - val_loss: 3.8545 - val_acc: 0.7796
Train on 2220 samples, validate on 1184 samples
Epoch 1/3
2219/2220 [============================>.] - ETA: 0s - loss: 0.0126 - acc: 0.9995
Epoch 00001: val_acc did not improve from 0.77956
2220/2220 [==============================] - 60s 27ms/sample - loss: 0.0126 - acc: 0.9995 - val_loss: 3.1633 - val_acc: 0.7340
Epoch 2/3
2219/2220 [============================>.] - ETA: 0s - loss: 0.0114 - acc: 0.9977
Epoch 00002: val_acc improved from 0.77956 to 0.78294, saving model to models\_All\_All#Face_spoofing300x2_17102019_0301[80, 120, 300][400].h5
2220/2220 [==============================] - 65s 29ms/sample - loss: 0.0114 - acc: 0.9

Train on 1881 samples, validate on 1184 samples
Epoch 1/3
1876/1881 [============================>.] - ETA: 0s - loss: 0.2040 - acc: 0.9925
Epoch 00001: val_acc did not improve from 0.78294
1881/1881 [==============================] - 33s 18ms/sample - loss: 0.2036 - acc: 0.9926 - val_loss: 5.7106 - val_acc: 0.7432
Epoch 2/3
1876/1881 [============================>.] - ETA: 0s - loss: 0.0349 - acc: 1.0000
Epoch 00002: val_acc did not improve from 0.78294
1881/1881 [==============================] - 33s 18ms/sample - loss: 0.0349 - acc: 1.0000 - val_loss: 8.4418 - val_acc: 0.7145
Epoch 3/3
1876/1881 [============================>.] - ETA: 0s - loss: 0.0243 - acc: 1.0000
Epoch 00003: val_acc did not improve from 0.78294
1881/1881 [==============================] - 33s 18ms/sample - loss: 0.0243 - acc: 1.0000 - val_loss: 7.6540 - val_acc: 0.7179
Train on 1979 samples, validate on 1184 samples
Epoch 1/3
1974/1979 [============================>.] - ETA: 0s - loss: 0.0123 - acc: 0.9980
Epoch

Epoch 3/3
2282/2284 [============================>.] - ETA: 0s - loss: 0.0027 - acc: 1.0000
Epoch 00003: val_acc did not improve from 0.80828
2284/2284 [==============================] - 39s 17ms/sample - loss: 0.0027 - acc: 1.0000 - val_loss: 3.3508 - val_acc: 0.7272
Train on 2177 samples, validate on 1184 samples
Epoch 1/3
2170/2177 [============================>.] - ETA: 0s - loss: 0.0165 - acc: 0.9977
Epoch 00001: val_acc did not improve from 0.80828
2177/2177 [==============================] - 59s 27ms/sample - loss: 0.0165 - acc: 0.9977 - val_loss: 2.1290 - val_acc: 0.6689
Epoch 2/3
2170/2177 [============================>.] - ETA: 0s - loss: 0.0098 - acc: 0.9995
Epoch 00002: val_acc did not improve from 0.80828
2177/2177 [==============================] - 59s 27ms/sample - loss: 0.0098 - acc: 0.9995 - val_loss: 1.9376 - val_acc: 0.6748
Epoch 3/3
2170/2177 [============================>.] - ETA: 0s - loss: 0.0043 - acc: 1.0000
Epoch 00003: val_acc did not improve from 0.80828
217

...

Train on 2363 samples, validate on 1184 samples
Epoch 1/3
2359/2363 [============================>.] - ETA: 0s - loss: 0.0014 - acc: 1.0000
Epoch 00001: val_acc did not improve from 0.80828
2363/2363 [==============================] - 40s 17ms/sample - loss: 0.0014 - acc: 1.0000 - val_loss: 4.4348 - val_acc: 0.6351
Epoch 2/3
2359/2363 [============================>.] - ETA: 0s - loss: 9.5039e-04 - acc: 1.0000
Epoch 00002: val_acc did not improve from 0.80828
2363/2363 [==============================] - 40s 17ms/sample - loss: 9.5017e-04 - acc: 1.0000 - val_loss: 4.8862 - val_acc: 0.6123
Epoch 3/3
2359/2363 [============================>.] - ETA: 0s - loss: 0.0016 - acc: 1.0000- ETA: 0s - loss: 0.0017 - acc
Epoch 00003: val_acc did not improve from 0.80828
2363/2363 [==============================] - 40s 17ms/sample - loss: 0.0016 - acc: 1.0000 - val_loss: 7.2719 - val_acc: 0.6292
Train on 1768 samples, validate on 1184 samples
Epoch 1/3
1764/1768 [============================>.] - ETA:

Epoch 2/3
2310/2317 [============================>.] - ETA: 0s - loss: 0.0299 - acc: 1.0000
Epoch 00002: val_acc did not improve from 0.81503
2317/2317 [==============================] - 62s 27ms/sample - loss: 0.0299 - acc: 1.0000 - val_loss: 1.4861 - val_acc: 0.7829
Epoch 3/3
2310/2317 [============================>.] - ETA: 0s - loss: 0.0220 - acc: 1.0000
Epoch 00003: val_acc did not improve from 0.81503
2317/2317 [==============================] - 62s 27ms/sample - loss: 0.0220 - acc: 1.0000 - val_loss: 1.6295 - val_acc: 0.7770
Train on 2317 samples, validate on 1184 samples
Epoch 1/3
2310/2317 [============================>.] - ETA: 0s - loss: 0.0112 - acc: 1.0000
Epoch 00001: val_acc did not improve from 0.81503
2317/2317 [==============================] - 39s 17ms/sample - loss: 0.0112 - acc: 1.0000 - val_loss: 8.7118 - val_acc: 0.6166
Epoch 2/3
2310/2317 [============================>.] - ETA: 0s - loss: 0.0085 - acc: 1.0000
Epoch 00002: val_acc did not improve from 0.81503
231

Train on 2421 samples, validate on 1184 samples
Epoch 1/3
2415/2421 [============================>.] - ETA: 0s - loss: 0.0155 - acc: 0.9979
Epoch 00001: val_acc did not improve from 0.83530
2421/2421 [==============================] - 65s 27ms/sample - loss: 0.0154 - acc: 0.9979 - val_loss: 2.1466 - val_acc: 0.6715
Epoch 2/3
2415/2421 [============================>.] - ETA: 0s - loss: 0.0136 - acc: 0.9988
Epoch 00002: val_acc did not improve from 0.83530
2421/2421 [==============================] - 65s 27ms/sample - loss: 0.0136 - acc: 0.9988 - val_loss: 1.8584 - val_acc: 0.7069
Epoch 3/3
2415/2421 [============================>.] - ETA: 0s - loss: 0.0094 - acc: 0.9992
Epoch 00003: val_acc did not improve from 0.83530
2421/2421 [==============================] - 65s 27ms/sample - loss: 0.0094 - acc: 0.9992 - val_loss: 1.9215 - val_acc: 0.7128
Train on 2421 samples, validate on 1184 samples
Epoch 1/3
2415/2421 [============================>.] - ETA: 0s - loss: 0.0113 - acc: 0.9996
Epoch

Epoch 3/3
2408/2415 [============================>.] - ETA: 0s - loss: 0.0085 - acc: 1.0000
Epoch 00003: val_acc did not improve from 0.83530
2415/2415 [==============================] - 41s 17ms/sample - loss: 0.0085 - acc: 1.0000 - val_loss: 3.4128 - val_acc: 0.6470
Train on 2377 samples, validate on 1184 samples
Epoch 1/3
2373/2377 [============================>.] - ETA: 0s - loss: 0.0208 - acc: 0.9983
Epoch 00001: val_acc did not improve from 0.83530
2377/2377 [==============================] - 64s 27ms/sample - loss: 0.0207 - acc: 0.9983 - val_loss: 1.2273 - val_acc: 0.8294
Epoch 2/3
2373/2377 [============================>.] - ETA: 0s - loss: 0.0143 - acc: 1.0000
Epoch 00002: val_acc did not improve from 0.83530
2377/2377 [==============================] - 64s 27ms/sample - loss: 0.0143 - acc: 1.0000 - val_loss: 1.3719 - val_acc: 0.8193
Epoch 3/3
2373/2377 [============================>.] - ETA: 0s - loss: 0.0144 - acc: 0.9996
Epoch 00003: val_acc did not improve from 0.83530
237

FileNotFoundError: [Errno 2] No such file or directory: 'database_serialized\\_All\\X_All.pickle'

In [6]:
person

'_All'